In [1]:
f = open('/home/her/DPPFAQ/project.txt', 'r')
x= f.read().splitlines()
f.close()

In [2]:
x1=" ".join(x)
x1=x1.split('.')
x1[:10]

['How much loan amount can I borrow in a Car Loan? You can borrow a loan amount from Rs50000 upto Rs30 Lakhs for Salaried Individuals and upto Rs1 Crore for Self-employed Individuals and Professionals depending on your eligibility basis the financial documents submitted, repayment capacity and loan amount required',
 ' RBL Bank only sanctions car loans for New Car purchase',
 ' How will I repay my Car Loan? We offer Car Loan as a term loan which can be repaid in easy EMIs (Equated Monthly Installments)',
 ' EMI can be paid through ECS facility, Standing Instruction to debit your RBL Bank account or through Post Dated Cheques with your EMI amount',
 ' EMI amount can be calculated using ‘EMI Calculator’ ',
 ' For how many months can I borrow a Car Loan? We offer convenient repayment options from 12 to 60 months',
 ' How much LTV can I get in a Car Loan? RBL Bank offers an LTV (Loan to Value) upto 90% of the value of the new car (includes Ex',
 ' Showroom Price + VAT + Insurance) What is 

In [3]:
sent=[]
counter=[]
a=0
for i in range(len(x1)):
    d=[]
    d.append(x1[i].split('?'))
    d=sum(d,[])
    if(len(d)>1):
        counter.append(i+a)
        a=a+1
    sent.append(list(d))
       

In [4]:
x=sum(sent,[])
x[:5]


['How much loan amount can I borrow in a Car Loan',
 ' You can borrow a loan amount from Rs50000 upto Rs30 Lakhs for Salaried Individuals and upto Rs1 Crore for Self-employed Individuals and Professionals depending on your eligibility basis the financial documents submitted, repayment capacity and loan amount required',
 ' RBL Bank only sanctions car loans for New Car purchase',
 ' How will I repay my Car Loan',
 ' We offer Car Loan as a term loan which can be repaid in easy EMIs (Equated Monthly Installments)']

#### REMOVING SPECIAL CHARACTERS

In [5]:
d =[]
elim=['1','2','3','4','5','6','7','8','9','0',':',';','>','<','/',',','-','\\',
      '#','%','(',')','+','"']
for i in x:
    sent = " ".join("".join(["" if ch in elim else ch for ch in i]).split())
    d.append(sent.lower())

In [6]:
counter[:5]

[0, 3, 7, 9, 11]

In [7]:
f=[]
for i in d:
#     print(len(i))
    if(len(i)!=0):
        f.append(i)
f[:5]
len(f)

99

In [8]:
f[:5]

['how much loan amount can i borrow in a car loan',
 'you can borrow a loan amount from rs upto rs lakhs for salaried individuals and upto rs crore for selfemployed individuals and professionals depending on your eligibility basis the financial documents submitted repayment capacity and loan amount required',
 'rbl bank only sanctions car loans for new car purchase',
 'how will i repay my car loan',
 'we offer car loan as a term loan which can be repaid in easy emis equated monthly installments']

#### SUBSET GROUPING

In [9]:
samplebound=counter[1:]

In [10]:
import numpy as np
bound=[]
bound.append([0,samplebound[0]-1])

for i in range(len(samplebound)-1):
    if ((samplebound[i+1]-samplebound[i])>1):
        a=[]
        a.append(samplebound[i])
        a.append(samplebound[i+1]-1)
        bound.append(a)
c=[]
c.append(a[1]+1)
c.append(len(f)-1)
bound.append(c)

subset=np.array(bound)


In [11]:
subset=np.array(subset)
np.save("/home/her/DPPFAQ/subsetfaq.npy", subset)
subset[:5]

array([[ 0,  2],
       [ 3,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12]])

#### wordvec

In [12]:
sentences=[]
for i in f:
    sentences.append((i).split())

In [13]:
type(sentences)
sentlength=[]
for i in sentences:
    sentlength.append(len(i))
print(max(sentlength))
sentences[:1]

38


[['how',
  'much',
  'loan',
  'amount',
  'can',
  'i',
  'borrow',
  'in',
  'a',
  'car',
  'loan']]

In [14]:
from gensim.models import Word2Vec
model = Word2Vec(sentences,size=300, min_count=1)
print(model)
words = list(model.wv.vocab)
model.save("/home/her/DPP/word2vecsubtitlesfaq.model")
# print((words))

Word2Vec(vocab=335, size=300, alpha=0.025)


#### Feature matrix

In [15]:
def sentencevector(sent):
    vector=[]
    c=[]
    for i in range(300):
        c.append(0)
    for word in sent:
           vector.append(model[word])
    for i in vector:
        c=c+np.array(i)
    return(c)


feature=[]
for i in range(len(sentences)):
    a=np.array(sentencevector(sentences[i]))
    feature.append(a)
    
features=np.array(feature)

/home/her/bhavya/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [16]:
type(feature)

list

In [17]:
np.save("/home/her/DPPFAQ/featurefaq.npy", feature)

In [18]:
import numpy as np
def fixed_point_map(feature,subset,subcheck):
        dim=len(feature[1])
        number=len(feature[:,1])
        feature=feature.reshape(number,dim)
        eta=.25
        exceptions=[]
        W=np.random.rand(dim,dim)
        Z = np.array([[0]*number]*number).astype(float)
        I = np.array([[0]*number]*number)
        for i in range(number):
            for j in range(number):
                if(i==j):
                    I[i][j]=1
                            
        subsetf=[]
        for i in subset:
            a=[]
            if(i[0]!=i[1]):
                for j in range(i[0],i[1]+1):
                    a.append(j)
                subsetf.append(a)
          
        L=np.matmul(np.matmul(feature,W),feature.transpose())
        

        for i in range(500):
            for k in range (len(subsetf)):
                sub=subsetf[k]
                ly=[]
                for i in sub:
                    a=[]
                    for j in sub:
                        a.append(L[i][j])
                    ly.append(a)
                try:  
                    ly=np.array(ly)
                    lyinv=np.linalg.inv(ly)
            
                    b=[]
                    for i in range (len(sub)):
                        for j in range (len(sub)):
                              b.append(lyinv[i][j])
                    start=0
                    stop=1

                    for i in sub:
                        for j in sub:
                            for k in range(start,stop):
                                Z[i][j]=Z[i][j]+b[k]
                                start=start+1
                                stop=stop+1
                                break
                except:
                    exceptions.append(k)
            Z=Z/len(subsetf)  
            e=np.matmul(feature.transpose(),np.matmul(Z-np.linalg.inv(L+I),feature))
            W=W+np.matmul(eta*(W),((np.matmul(e,W))))
            L=np.matmul(np.matmul(feature,W),feature.transpose())
        probab=[]
        
        for k in range (len(subsetf)):
            sub=subsetf[k]
            ly=[]
            for i in sub:
                a=[]
                for j in sub:
                    a.append(L[i][j])
                ly.append(a)
            ly=np.array(ly)
            probab.append((np.linalg.det(ly)/np.linalg.det(L+I)))
        
        checkprobab=[]    
        
        for k in range (len(subcheck)):
            sub=subcheck[k]
            ly=[]
            for i in sub:
                a=[]
                for j in sub:
                    a.append(L[i][j])
                ly.append(a)
            ly=np.array(ly)
            checkprobab.append((np.linalg.det(ly)/np.linalg.det(L+I))) 
        
        return L,exceptions,probab,checkprobab
feature=np.load("/home/her/DPPFAQ/featurefaq.npy")
subset=np.load("/home/her/DPPFAQ/subsetfaq.npy")
subcheck=[[0,3],[3,7],[7,9],[9,11],[11,12],[12],[13],[12,13],[60,61,62,63,64]]
L,exceptions,probab,checkprobab=fixed_point_map(feature,subset,subcheck)

In [19]:
import numpy as np
def fixed_point_map(feature,subset,subcheck):
        dim=len(feature[1])
        number=len(feature[:,1])
        feature=feature.reshape(number,dim)
        eta=.25
        exceptions=[]
        W=np.random.rand(dim,dim)
        Z = np.array([[0]*number]*number).astype(float)
        I = np.array([[0]*number]*number)
        for i in range(number):
            for j in range(number):
                if(i==j):
                    I[i][j]=1
                            
        subsetf=[]
        for i in subset:
            a=[]
            if(i[0]!=i[1]):
                for j in range(i[0],i[1]+1):
                    a.append(j)
                subsetf.append(a)
          
        L=np.matmul(feature,feature.transpose())
        

        for i in range(500):
            for k in range (len(subsetf)):
                sub=subsetf[k]
                ly=[]
                for i in sub:
                    a=[]
                    for j in sub:
                        a.append(L[i][j])
                    ly.append(a)
                try:  
                    ly=np.array(ly)
                    lyinv=np.linalg.inv(ly)
            
                    b=[]
                    for i in range (len(sub)):
                        for j in range (len(sub)):
                              b.append(lyinv[i][j])
                    start=0
                    stop=1

                    for i in sub:
                        for j in sub:
                            for k in range(start,stop):
                                Z[i][j]=Z[i][j]+b[k]
                                start=start+1
                                stop=stop+1
                                break
                except:
                    exceptions.append(k)
            Z=Z/len(subsetf)  
            e=Z-np.linalg.inv(L+I)
            L=L+np.matmul(eta*(L),((np.matmul(e,L))))
        probab=[]
        
        for k in range (len(subsetf)):
            sub=subsetf[k]
            ly=[]
            for i in sub:
                a=[]
                for j in sub:
                    a.append(.5*L[i][j])
                ly.append(a)
            ly=np.array(ly)
            probab.append((np.linalg.det(ly)/np.linalg.det(L+I)))
        
        checkprobab=[]    
        
        for k in range (len(subcheck)):
            sub=subcheck[k]
            ly=[]
            for i in sub:
                a=[]
                for j in sub:
                    a.append(L[i][j])
                ly.append(a)
            ly=np.array(ly)
            checkprobab.append((np.linalg.det(ly)/np.linalg.det(L+I))) 
        
        return L,exceptions,probab,checkprobab
feature=np.load("/home/her/DPPFAQ/featurefaq.npy")
subset=np.load("/home/her/DPPFAQ/subsetfaq.npy")
subcheck=[[0,3],[3,7],[7,9],[9,11],[11,12],[12],[13],[12,13],[60,61,62,63,64]]
L,exceptions,probab,checkprobab=fixed_point_map(feature,subset,subcheck)

In [20]:
np.absolute(probab)

array([1.91228166e-06, 1.68437389e-08, 4.58615123e-06, 1.96274167e-05,
       1.96750406e-05, 2.35145662e-06, 2.79312168e-11, 1.60920185e-08,
       2.10310155e-07, 9.45668481e-08, 4.41230272e-05, 4.75288483e-07,
       1.25839949e-08, 1.36000546e-08, 5.36324147e-06, 5.53347744e-05,
       2.34252520e-06, 3.05587799e-13, 1.22540086e-08, 5.02884751e-06,
       1.10506517e-10, 3.34687720e-07, 3.93178318e-11, 3.57429600e-06,
       4.87838051e-09, 5.35217977e-09, 6.47810816e-09, 1.44741384e-05,
       1.63376768e-08])

In [22]:
subset

array([[ 0,  2],
       [ 3,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12],
       [13, 15],
       [16, 21],
       [22, 25],
       [26, 28],
       [29, 32],
       [33, 34],
       [35, 37],
       [38, 41],
       [42, 45],
       [46, 47],
       [48, 49],
       [50, 52],
       [53, 59],
       [60, 63],
       [64, 65],
       [66, 70],
       [71, 73],
       [74, 78],
       [79, 80],
       [81, 84],
       [85, 88],
       [89, 92],
       [93, 94],
       [95, 98]])

In [18]:
subcheck=[[0,1,2],[0,1,2,3],[3,4,5,6],[7,8,9],[9,11],[11,13],[13,16],
          [16,22],[1,2],[2,3]]
checkprobab=[]    
        
number =99
I = np.array([[0]*number]*number)
for i in range(number):
    for j in range(number):
        if(i==j):
            I[i][j]=1
                    
for k in range (len(subcheck)):
    sub=subcheck[k]
    ly=[]
    for i in sub:
        a=[]
        for j in sub:
            a.append(L[i][j])
        ly.append(a)
    ly=np.array(ly)
    checkprobab.append((np.linalg.det(ly)/np.linalg.det(L+I))) 
np.absolute(checkprobab)

array([2.25449203e-05, 1.13307837e-07, 4.71339068e-07, 3.65763528e-07,
       1.22076204e-05, 3.90230019e-06, 4.97368923e-06, 6.91201498e-06,
       5.15482911e-06, 8.82123539e-06])

In [21]:
L=[]
for i in range(number):
            for j in range(number):    
                M=np.exp((-1*0.25*(np.linalg.norm((np.array(feature[i]))
                                                  -np.array(feature[j]),2))))
                L.append(M)
L=np.array(L)
L=L.reshape(number,number)

NameError: name 'number' is not defined